In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.externals import joblib
from google.colab import drive 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score
import tensorflow_hub as hub
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Embedding
def Embedding(X):
  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  X = embed(X)
  X = X.numpy()
  return X

In [ ]:
## Lectura de datos
data = pd.read_csv('/content/drive/My Drive/Factored project/Models/newdata.csv')
data = data.drop(columns=['Unnamed: 0'], axis=1)
Y = data['target']
X = data.drop(columns=['target'], axis=1)

In [ ]:
## Modelo
import lightgbm as lgb
model = lgb.LGBMClassifier(random_state=40)

In [ ]:
#dist = dict(n_estimators=[70, 80, 90, 100, 110, 120, 130], num_leaves=[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], max_depth=[4, 5, 6, 7, 8, 9, 10], learning_rate=[0.01, 0.05, 0.09, 0.1, 0.5, 0.9, 1])
#RS_light = RandomizedSearchCV(model, dist, random_state=0, scoring='f1')
Resultados = {'Set':['Train', 'Test']}
for i in X.columns:
  x_em = Embedding(X.loc[:,i])
  # Partición de los datos
  x_train, x_test, y_train, y_test = train_test_split(x_em, Y, test_size=0.2, random_state=40)
  # Buscar hiperparámetros que maximizan la metrica f1
  var = 'model_lightgbm_{}'.format(i)
  search = joblib.load('/content/drive/My Drive/Factored project/Hyperparameter/{}.pkl'.format(var))
  y_pre = search.predict(x_test)
  print(search.best_params_)
  Resultados[i] = [search.best_score_, f1_score(y_test, y_pre)]

{'num_leaves': 74, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1}
{'num_leaves': 73, 'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1}
{'num_leaves': 74, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1}
{'num_leaves': 73, 'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1}
{'num_leaves': 74, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1}
{'num_leaves': 74, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1}
{'num_leaves': 74, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1}


In [ ]:
R_hyp = pd.DataFrame(data=Resultados)
R_hyp.to_csv('/content/drive/My Drive/Factored project/Hyperparameter/R_Lightgbm.csv')
R_hyp.head()

,Set,text,NoRuido,PRE,URLS,HTML,EMOJIS,TODO
0,Train,0.770267,0.763013,0.758683,0.763013,0.770267,0.770267,0.758683
1,Test,0.790514,0.782051,0.759113,0.782051,0.790514,0.790514,0.759113
